<a href="https://colab.research.google.com/github/nazike/dataoptions/blob/main/OptionsAnalysisPricing191021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#read file from the github repo
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
df1 = pd.read_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/V2.csv')
df1.columns = columns
df2 = pd.read_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/data2.csv')
df2.columns = columns

In [3]:
#merge two dataframes
df = pd.concat([df1, df2], axis=0)


In [13]:
df.describe()
#dataframe to csv
#df.to_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/fulldataoptions.csv', index=False)

In [59]:
# Read the data
X_full = df.copy()
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
X_full.columns = columns
X_full['target'] = (X_full['am_price']-X_full['eu_price']).copy()

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['target'], inplace=True)
y = X_full.target
y_amer = X_full['am_price']
X_full.drop(['am_price'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/fulldataoptions.csv', index=False)
X_full.drop(['target'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/fulldataoptionstest.csv', index=False)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality 
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()



In [57]:
len(X_valid)

3806

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
#model = XGBRegressor(n_estimators=5000, random_state=0, learning_rate=0.07, n_jobs=4)

model = XGBRegressor(max_depth=6, reg_alpha=0.0002433756223211188, colsample_bytree=0.8242913620513932, learning_rate=0.05640249301683806, gamma=4.5885092732685184e-05,
                        reg_lambda=0.014400178832201075, min_child_weight=0.003963394614607396, 
                        n_estimators=10000, n_jobs=4, random_state=0, subsample=0.8586706877598019)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)



[17:32:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['asset_price', 'maturity',
                                                   'rate', 'div', 'ivol',
                                                   'eu_price']),
 

In [46]:
# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MSE',mean_squared_error(y_valid, preds))

print('MAE:', mean_absolute_error(y_valid, preds))

MSE 0.018818651598030145
MAE: 0.055486571399716585


In [71]:
preds_full=clf.predict(X_full)
preds_full

array([ 0.23187047,  0.01266921,  0.00729606, ...,  0.00419331,
        0.09903696, -0.00023282], dtype=float32)

In [75]:
#create a dataframe with the predicted values and write it to csv   
predicted_df=df.copy()
predicted_df['predicted_target']=preds_full
predicted_df['predicted_am_price']=predicted_df['predicted_target']+predicted_df['eu_price']
predicted_df.to_csv('/Users/nazymazimbayev/Desktop/VS Code projects/dataoptions/predicted_dataoptions.csv', index=False)

In [104]:
#calculation of the neural net time 
X_sample = X_full.iloc[0:1].copy()


import timeit
def test_nn():
    return clf.predict(X_sample)

loop = 1000

result_nn = timeit.timeit('test_nn()', globals=globals(), number=loop)
print(result_nn, 'seconds per loop times of test function')
print(result_nn/loop, 'seconds per each clf.predict()')

3.008754445996601 seconds per loop times of test function
0.003008754445996601 seconds per each clf.predict()


In [96]:
#function to calculate binomial option price

import numpy as np

def binomial_put(S, K, T, R, div, sig, n):
    
    h = T/n
    Rinv = np.exp(-R*h)
    a = np.exp((R-div)*h)
    b2 = (a**2)*(np.exp(h*sig**2)-1)
    tmp = a**2 + b2 + 1
    u = (tmp + (tmp**2 - 4*a**2)**0.5)/(2*a)
    d = 1/u
    p = (a-d)/(u-d)
    q = 1-p
    pp = Rinv*p
    qq = Rinv*q
    x = 2*n + 1
    
    s = np.zeros(x)
    s[n] = S
    
    for j in range(1,n+1):
        s[n+j] = s[n-1+j]*u
        s[n-j] = s[n+1-j]*d
        
    v = np.zeros(x)
    
    for j in range(1,x+2,2):
        v[j-1] = max(K-s[j-1],0)
        
    for i in range(n-1,0,-1):
        for j in range(-i,i+2,2):
            j1 = j+n+1
            v[j1-1] = max(K-s[j1-1],pp*v[j1]+qq*v[j1-2])
    
    y=pp*v[n+1]+qq*v[n-1]
    
    return y

In [101]:
#parameters for the binomial put option price calcualtion , similar to X_full.iloc[0:1]

S = 91.063192
K = 100.0
T = 0.64276
R = 0.073981	
div = 0.069325	
sig = 0.312864
n=250


result = binomial_put(S, K, T, R, div, sig, n)


In [102]:
#binomial option price calcualtion time
def test_bt():
    return binomial_put(S, K, T, R, div, sig, n)

loop = 1000

result_bt = timeit.timeit('test_bt()', globals=globals(), number=loop)
print(result_bt, 'seconds per loop times of test function')
print(result_bt/loop, 'seconds per each binomial_put')

30.53690537600778 seconds per loop times of test function
0.03053690537600778 seconds per each binomial_put


In [103]:
#result, which shows how much faster the neural net is than the binomial tree
print(result_bt/result_nn, 'times nn faster than bt')

9.488753351587496 times nn faster than bt
